In [2]:
import pandas as pd
import matplotlib.pyplot as plt
%matplotlib inline
import pystan
import scipy

In [3]:
df = pd.read_excel("/Users/apple/Downloads/userVisits.xlsx")
df.head()

,user_id,total_visits,weeklyVisits,Interval_visits_int
0,1332384,70,"0,0,0,1,2,1,1,0,0,2,1,0,0,2,1,0,0,0,1,1,2,1,0,...","[3, 0, 0, 0, 2, 0, 2, 0, 3, 0, 0, 0, 1, 4, 2, ..."
1,1332636,4,"0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...","[62, 12, 1, 28]"
2,1334543,4,"1,3,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...","[6, 0]"
3,1336195,1,"0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...",[113]
4,1339702,16,"1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,...","[110, 14, 29, 7, 29, 2, 2, 1, 0, 2, 4, 1, 1, 1..."


In [4]:
df = df.iloc[df.index <= 200]

In [5]:
visits = df['weeklyVisits'].tolist()

In [6]:
visits_int = list(list(int(a) for a in (str(r).split(',')))for r in visits)

In [7]:
len(visits_int)

201

In [8]:
len(visits_int[10])

113

In [9]:
visits_code = """
data {
    int<lower=0> J;// number of users
    int<lower=0> W;// number of weeks
    int y[J,W]; // estimated number of visits next week
    //real<lower=0> sigma[J]; // s.e. of visits estimates
}
parameters {
    real mu;
    real<lower=0> tau;
    real<lower=0> eta[J];
    //real sigma;
}
transformed parameters {
     real<lower=0> theta[J];
     for (j in 1:J)
     theta[j]= eta[j];
}
model {
    //tau ~ normal(0,1);
    for(j in 1:J)
    eta[j] ~ gamma(1,2);
    for (j in 1:J){
    for(w in 1:W){
    y[j,w] ~ poisson(theta[j]);
    }
 }
}
"""



visits_dat = {'J': len(visits_int),
              'W':113,
              
              'y': visits_int
               }



sm = pystan.StanModel(model_code=visits_code)
fit = sm.sampling(data=visits_dat, iter=1000, chains=4)

INFO:pystan:COMPILING THE C++ CODE FOR MODEL anon_model_ff24efa5eea27f78253b71a2ef5bfd0c NOW.


In [10]:
fit


/Users/apple/anaconda/lib/python3.6/site-packages/numpy/core/_methods.py:70: RuntimeWarning: overflow encountered in reduce
  ret = umr_sum(arr, axis, dtype, out, keepdims)
/Users/apple/anaconda/lib/python3.6/site-packages/numpy/core/_methods.py:102: RuntimeWarning: overflow encountered in reduce
  arrmean = umr_sum(arr, axis, dtype, keepdims=True)


Inference for Stan model: anon_model_ff24efa5eea27f78253b71a2ef5bfd0c.
4 chains, each with iter=1000; warmup=500; thin=1; 
post-warmup draws per chain=500, total post-warmup draws=2000.

             mean se_mean     sd   2.5%    25%    50%    75%  97.5%  n_eff   Rhat
mu          -5.38   18.85  26.66 -61.37 -18.68   3.72   9.51  32.55      2   5.04
tau           inf     inf    inf4.0e3063.9e3078.4e3071.3e3081.8e308   2000    nan
eta[0]       0.63  8.5e-3   0.07   0.49   0.58   0.63   0.67   0.77     72   1.05
eta[1]       0.04  5.2e-3   0.02   0.01   0.02   0.03   0.05   0.08     15   1.27
eta[2]       0.03  7.5e-3   0.02   0.01   0.02   0.03   0.04   0.11      7   1.38
eta[3]       0.01  1.6e-3 9.1e-3 3.2e-3 8.3e-3   0.01   0.02   0.04     32   1.07
eta[4]       0.15  5.3e-3   0.03    0.1   0.13   0.15   0.17   0.24     38    1.1
eta[5]       0.14  9.3e-3   0.03   0.08   0.11   0.13   0.16    0.2     13   1.35
eta[6]       0.06  3.9e-3   0.02   0.03   0.04   0.05   0.07    0.1     25 

In [ ]:
fit.plot()